In [1]:
using Mamba

INFO: Precompiling module Compose...
INFO: Precompiling module Gadfly...
INFO: Recompiling stale cache file /Users/NIGG/.julia/lib/v0.4/DataFrames.ji for module DataFrames.
INFO: Recompiling stale cache file /Users/NIGG/.julia/lib/v0.4/KernelDensity.ji for module KernelDensity.


In [2]:
model = Model(

  y = Stochastic(1,
    (mu, s2) ->  MvNormal(mu, sqrt(s2)),
    false
  ),

  mu = Logical(1,
    (xmat, beta) -> xmat * beta,
    false
  ),

  beta = Stochastic(1,
    () -> MvNormal(2, sqrt(1000))
  ),

  s2 = Stochastic(
    () -> InverseGamma(0.001, 0.001)
  )

)

Object of type "Mamba.Model"
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "Mamba.ScalarStochastic"
NaN
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element Mamba.ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
mu:
An unmonitored node of type "0-element Mamba.ArrayLogical{1}"
Float64[]


In [3]:
# Case 1: Multivariate Normal with independence covariance matrix
beta = Stochastic(1,
  () -> MvNormal(2, sqrt(1000))
)

# Case 2: One common univariate Normal
beta = Stochastic(1,
  () -> Normal(0, sqrt(1000))
)

# Case 3: Array of univariate Normals
beta = Stochastic(1,
  () -> UnivariateDistribution[Normal(0, sqrt(1000)), Normal(0, sqrt(1000))]
)

# Case 4: Array of univariate Normals
beta = Stochastic(1,
  () -> UnivariateDistribution[Normal(0, sqrt(1000)) for i in 1:2]
)

0-element Mamba.ArrayStochastic{1}

In [4]:
## Hybrid No-U-Turn and Slice Sampling Scheme
scheme1 = [NUTS(:beta),
           Slice(:s2, 3.0)]

## No-U-Turn Sampling Scheme
scheme2 = [NUTS([:beta, :s2])]

1-element Array{Mamba.Sampler{Mamba.NUTSTune},1}:
 An object of type "Mamba.Sampler{Mamba.NUTSTune}"
Sampling Block Nodes:
[:beta,:s2]

AST(:($(Expr(:lambda, Any[:(model::Model),:(block::Integer)], Any[Any[Any[:model,:Any,18],Any[:block,:Any,18],Any[:f,:Any,18]],Any[],0,Any[]], :(begin 
        model = (top(typeassert))(model,Mamba.Model)
        block = (top(typeassert))(block,Mamba.Integer)
        f = (anonymous function)
        return f(model,block)
    end)))))


In [5]:
## User-Defined Samplers

Gibbs_beta = Sampler([:beta],
  (beta, s2, xmat, y) ->
    begin
      beta_mean = mean(beta.distr)
      beta_invcov = invcov(beta.distr)
      Sigma = inv(xmat' * xmat / s2 + beta_invcov)
      mu = Sigma * (xmat' * y / s2 + beta_invcov * beta_mean)
      rand(MvNormal(mu, Sigma))
    end
)

Gibbs_s2 = Sampler([:s2],
  (mu, s2, y) ->
    begin
      a = length(y) / 2.0 + shape(s2.distr)
      b = sumabs2(y - mu) / 2.0 + scale(s2.distr)
      rand(InverseGamma(a, b))
    end
)

## User-Defined Sampling Scheme
scheme3 = [Gibbs_beta, Gibbs_s2]

2-element Array{Mamba.Sampler{Dict{Any,Any}},1}:
 An object of type "Mamba.Sampler{Dict{Any,Any}}"
Sampling Block Nodes:
[:beta]

AST(:($(Expr(:lambda, Any[:(model::Model),:(block::Integer)], Any[Any[Any[:model,:Any,18],Any[:block,:Any,18],Any[:f,:Any,18]],Any[],0,Any[]], :(begin 
        model = (top(typeassert))(model,Mamba.Model)
        block = (top(typeassert))(block,Mamba.Integer)
        f = (anonymous function)
        return f((Mamba.getindex)(model,:beta),(Mamba.getindex)(model,:s2),(Mamba.getindex)(model,:xmat),(Mamba.getindex)(model,:y))
    end)))))

 An object of type "Mamba.Sampler{Dict{Any,Any}}"
Sampling Block Nodes:
[:s2]

AST(:($(Expr(:lambda, Any[:(model::Model),:(block::Integer)], Any[Any[Any[:model,:Any,18],Any[:block,:Any,18],Any[:f,:Any,18]],Any[],0,Any[]], :(begin 
        model = (top(typeassert))(model,Mamba.Model)
        block = (top(typeassert))(block,Mamba.Integer)
        f = (anonymous function)
        return f((Mamba.getindex)(model,:mu),(Mamba.getind

In [6]:
## Graph Representation of Nodes

>>> draw(model)

digraph MambaModel {
  "mu" [shape="diamond", fillcolor="gray85", style="filled"];
    "mu" -> "y";
  "xmat" [shape="box", fillcolor="gray85", style="filled"];
    "xmat" -> "mu";
  "beta" [shape="ellipse"];
    "beta" -> "mu";
  "s2" [shape="ellipse"];
    "s2" -> "y";
  "y" [shape="ellipse", fillcolor="gray85", style="filled"];
}

>>> draw(model, filename="lineDAG.dot")

LoadError: LoadError: syntax: ">>>" is not a unary operator
while loading In[6], in expression starting on line 3

In [10]:
using GraphViz


In [11]:
display(Graph(graph2dot(model)))

Error: Layout type: "neato" not recognized. Use one of: circo dot fdp neato nop nop1 nop2 osage patchwork sfdp twopi
Error: Layout was not done
Error: Layout was not done


In [12]:
## Data
line = Dict{Symbol, Any}(
  :x => [1, 2, 3, 4, 5],
  :y => [1, 3, 3, 3, 5]
)
line[:xmat] = [ones(5) line[:x]]

5x2 Array{Float64,2}:
 1.0  1.0
 1.0  2.0
 1.0  3.0
 1.0  4.0
 1.0  5.0

In [13]:
## Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => rand(Normal(0, 1), 2),
    :s2 => rand(Gamma(1, 1))
  )
for i in 1:3
]

3-element Array{Dict{Symbol,Any},1}:
 Dict{Symbol,Any}(:y=>[1,3,3,3,5],:s2=>0.36645899740731525,:beta=>[0.5610846997912224,1.367308899146061]) 
 Dict{Symbol,Any}(:y=>[1,3,3,3,5],:s2=>5.3316567000392,:beta=>[0.44521596790502194,-0.8789350648786844])  
 Dict{Symbol,Any}(:y=>[1,3,3,3,5],:s2=>1.4176932425242237,:beta=>[1.0365520530764907,-0.9222041999405783])

In [14]:
## MCMC Simulations

setsamplers!(model, scheme1)
sim1 = mcmc(model, line, inits, 10000, burnin=250, thin=2, chains=3)

setsamplers!(model, scheme2)
sim2 = mcmc(model, line, inits, 10000, burnin=250, thin=2, chains=3)

setsamplers!(model, scheme3)
sim3 = mcmc(model, line, inits, 10000, burnin=250, thin=2, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:40:12 of 0:40:15 remaining]
Chain 1:  10% [0:00:31 of 0:00:35 remaining]
Chain 1:  20% [0:00:17 of 0:00:21 remaining]
Chain 1:  30% [0:00:12 of 0:00:17 remaining]
Chain 1:  40% [0:00:09 of 0:00:14 remaining]
Chain 1:  50% [0:00:07 of 0:00:13 remaining]
Chain 1:  60% [0:00:05 of 0:00:12 remaining]
Chain 1:  70% [0:00:04 of 0:00:12 remaining]
Chain 1:  80% [0:00:02 of 0:00:11 remaining]
Chain 1:  90% [0:00:01 of 0:00:11 remaining]
Chain 1: 100% [0:00:00 of 0:00:10 remaining]

Chain 2:   0% [0:00:09 of 0:00:09 remaining]
Chain 2:  10% [0:00:06 of 0:00:07 remaining]
Chain 2:  20% [0:00:10 of 0:00:12 remaining]
Chain 2:  30% [0:00:08 of 0:00:11 remaining]
Chain 2:  40% [0:00:06 of 0:00:10 remaining]
Chain 2:  50% [0:00:05 of 0:00:10 remaining]
Chain 2:  60% [0:00:04 of 0:00:09 remaining]
Chain 2:  70% [0:00:03 of 0:00:09 remaining]
Chain 2:  80% [0:00:02 of 0:00:09 remaining]
Chain 2:  90% [0:00:01 of 0:00:10 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 252:10000
Thinning interval = 2
Chains = 1,2,3
Samples per chain = 4875

4875x3x3 Array{Float64,3}:
[:, :, 1] =
 -0.420409   0.966174  1.72495 
 -2.62584    1.98674   7.10747 
  0.184019   1.11658   0.457038
  1.10798    0.708448  0.592361
  1.89388    0.898197  1.69327 
  0.645553   0.878223  0.119913
 -0.414345   1.08854   0.493007
  0.0181287  0.914951  0.352161
  1.0639     0.589027  1.02145 
  0.748179   0.701935  2.89631 
 -0.643133   0.706209  4.24702 
  1.52024    0.646727  0.690471
  0.363537   0.696997  1.78172 
  ⋮                            
 -0.5965     1.26881   1.50217 
  1.38572    0.555554  0.360549
  1.00497    0.730194  1.64877 
  1.12806    0.756175  0.275413
  0.5713     0.870993  0.205924
  1.11759    0.613436  0.220883
  0.0203204  0.817948  0.85294 
 -0.661105   1.08493   0.511444
  1.41432    0.763788  1.28015 
  0.932881   0.703351  0.190952
  2.14813    0.336995  0.40045 
  0.0957242  0.925747  0.336593

[:, :,

In [15]:
gelmandiag(sim1, mpsrf=true, transform=true) |> showall

Iterations = 252:10000
Thinning interval = 2
Chains = 1,2,3
Samples per chain = 4875

Gelman, Rubin, and Brooks Diagnostic:
              PSRF 97.5%
     beta[1] 1.004 1.004
     beta[2] 1.005 1.005
          s2 1.006 1.013
Multivariate 1.005   NaN



In [16]:
gewekediag(sim1) |> showall

Iterations = 252:10000
Thinning interval = 2
Chains = 1,2,3
Samples per chain = 4875

Geweke Diagnostic:
First Window Fraction = 0.1
Second Window Fraction = 0.5

        Z-score p-value
beta[1]   0.135  0.8922
beta[2]  -0.342  0.7320
     s2   1.700  0.0891

        Z-score p-value
beta[1]   0.415  0.6781
beta[2]  -0.299  0.7647
     s2  -1.504  0.1327

        Z-score p-value
beta[1]  -0.257  0.7975
beta[2]   0.834  0.4043
     s2  -1.236  0.2165



In [17]:
heideldiag(sim1) |> showall

Iterations = 252:10000
Thinning interval = 2
Chains = 1,2,3
Samples per chain = 4875

Heidelberger and Welch Diagnostic:
Target Halfwidth Ratio = 0.1
Alpha = 0.05

        Burn-in Stationarity p-value    Mean     Halfwidth  Test
beta[1]     251            1  0.4857  0.6271229 0.059607943    1
beta[2]     251            1  0.5823  0.7932595 0.017043466    1
     s2     251            1  0.0900  1.1893637 0.181081018    0

        Burn-in Stationarity p-value    Mean     Halfwidth  Test
beta[1]     251            1  0.9097  0.6088265 0.066213288    0
beta[2]     251            1  0.9391  0.7989795 0.018619125    1
     s2    1225            1  0.2215  1.1332336 0.194114662    0

        Burn-in Stationarity p-value    Mean     Halfwidth  Test
beta[1]     251            1  0.5733 0.59859523 0.066334119    0
beta[2]     251            1  0.3998 0.80441214 0.019121652    1
     s2     251            1  0.6029 1.51332099 0.709998937    0



In [18]:
rafterydiag(sim1) |> showall

Iterations = 252:10000
Thinning interval = 2
Chains = 1,2,3
Samples per chain = 4875

Raftery and Lewis Diagnostic:
Quantile (q) = 0.025
Accuracy (r) = 0.005
Probability (s) = 0.95

        Thinning Burn-in    Total   Nmin Dependence Factor
beta[1]        2     271 2.1459×10⁴ 3746         5.7285104
beta[2]        4     271 2.4215×10⁴ 3746         6.4642285
     s2        2     257 8.2730×10³ 3746         2.2084891

        Thinning Burn-in    Total   Nmin Dependence Factor
beta[1]        6     275 2.6597×10⁴ 3746         7.1001068
beta[2]        4     267 1.9679×10⁴ 3746         5.2533369
     s2        2     257 8.6890×10³ 3746         2.3195408

        Thinning Burn-in    Total   Nmin Dependence Factor
beta[1]        2     267 1.7897×10⁴ 3746         4.7776295
beta[2]        4     271 2.4775×10⁴ 3746         6.6137213
     s2        2     257 8.2730×10³ 3746         2.2084891



In [19]:
describe(sim1)

Iterations = 252:10000
Thinning interval = 2
Chains = 1,2,3
Samples per chain = 4875

Empirical Posterior Estimates:
           Mean        SD       Naive SE       MCSE        ESS   
beta[1] 0.61151487 1.24763621 0.0103166817 0.016697744 4875.00000
beta[2] 0.79888373 0.38007294 0.0031428164 0.004719440 4875.00000
     s2 1.49680310 2.88662973 0.0238694902 0.187938681  235.91188

Quantiles:
            2.5%        25.0%       50.0%     75.0%     97.5% 
beta[1] -1.972107794 0.009179614 0.59713024 1.2207792 3.092993
beta[2]  0.040674834 0.621236739 0.80190294 0.9807751 1.566160
     s2  0.173057759 0.391739202 0.68072071 1.3485452 8.864448



In [20]:
hpd(sim1)

          95% Lower  95% Upper
beta[1] -1.973328999 3.0916141
beta[2]  0.006852965 1.5179636
     s2  0.073655658 5.1566056



In [21]:
cor(sim1)

           beta[1]      beta[2]        s2     
beta[1]  1.000000000 -0.901780218  0.017214835
beta[2] -0.901780218  1.000000000 -0.029389504
     s2  0.017214835 -0.029389504  1.000000000



In [22]:
autocor(sim1)

           Lag 2       Lag 10       Lag 20       Lag 100   
beta[1] 0.29533458  -0.051111514 -0.017586327 -0.0131754123
beta[2] 0.24282346  -0.044766491 -0.042868878 -0.0155397097
     s2 0.80240506   0.341014367  0.090544986  0.0154897833

           Lag 2       Lag 10       Lag 20       Lag 100   
beta[1] 0.22007187   0.011872593  -0.06511239  -0.017041782
beta[2] 0.18587338   0.043978609  -0.06927239  -0.019477807
     s2 0.96061224   0.887534805   0.83600567   0.322968387

           Lag 2       Lag 10       Lag 20       Lag 100   
beta[1] 0.27125833  0.0014862231 0.0029120077  -0.019160270
beta[2] 0.23685932 -0.0142373274 0.0279196839  -0.014282321
     s2 0.93619518  0.7768844948 0.6697718566   0.011127104



In [23]:
changerate(sim1)

             Change Rate
     beta[1]       0.869
     beta[2]       0.869
          s2       1.000
Multivariate       1.000



In [24]:
dic(sim1)

MCMC Processing of 4875 Iterations x 3 Chains...

Chain 1:   0% [0:01:59 of 0:01:60 remaining]
Chain 1:  10% [0:00:02 of 0:00:03 remaining]
Chain 1:  20% [0:00:01 of 0:00:01 remaining]
Chain 1:  30% [0:00:01 of 0:00:01 remaining]
Chain 1:  40% [0:00:00 of 0:00:01 remaining]
Chain 1:  50% [0:00:00 of 0:00:01 remaining]
Chain 1:  60% [0:00:00 of 0:00:01 remaining]
Chain 1:  70% [0:00:00 of 0:00:00 remaining]
Chain 1:  80% [0:00:00 of 0:00:00 remaining]
Chain 1:  90% [0:00:00 of 0:00:00 remaining]
Chain 1: 100% [0:00:00 of 0:00:00 remaining]

Chain 2:   0% [0:03:13 of 0:03:13 remaining]
Chain 2:  10% [0:00:04 of 0:00:04 remaining]
Chain 2:  20% [0:00:02 of 0:00:02 remaining]
Chain 2:  30% [0:00:01 of 0:00:01 remaining]
Chain 2:  40% [0:00:01 of 0:00:01 remaining]
Chain 2:  50% [0:00:00 of 0:00:01 remaining]
Chain 2:  60% [0:00:00 of 0:00:01 remaining]
Chain 2:  70% [0:00:00 of 0:00:01 remaining]
Chain 2:  80% [0:00:00 of 0:00:01 remaining]
Chain 2:  90% [0:00:00 of 0:00:01 remaining]
Chai

      DIC    Effective Parameters
pD 13.725497           0.72513885
pV 26.232944           6.97886252



Chain 3:  40% [0:00:01 of 0:00:01 remaining]
Chain 3:  50% [0:00:01 of 0:00:01 remaining]
Chain 3:  60% [0:00:00 of 0:00:01 remaining]
Chain 3:  70% [0:00:00 of 0:00:01 remaining]
Chain 3:  80% [0:00:00 of 0:00:01 remaining]
Chain 3:  90% [0:00:00 of 0:00:01 remaining]
Chain 3: 100% [0:00:00 of 0:00:01 remaining]



In [25]:
aic(sim1)

LoadError: LoadError: UndefVarError: aic not defined
while loading In[25], in expression starting on line 1

In [27]:
sim = sim1[1000:5000, ["beta[1]", "beta[2]"], :]
describe(sim)

Iterations = 1000:5000
Thinning interval = 2
Chains = 1,2,3
Samples per chain = 2001

Empirical Posterior Estimates:
           Mean       SD       Naive SE      MCSE       ESS   
beta[1] 0.6072372 1.26739343 0.016357890 0.031433774 1625.6605
beta[2] 0.8008618 0.38293058 0.004942377 0.008557990 2001.0000

Quantiles:
            2.5%         25.0%      50.0%     75.0%     97.5%  
beta[1] -2.048665709 0.0042377046 0.5935937 1.2268551 3.0559452
beta[2]  0.061439738 0.6169185381 0.8048024 0.9808134 1.5662691



In [28]:
## Write to and Read from an External File
write("sim1.jls", sim1)
sim1 = read("sim1.jls", ModelChains)

Object of type "Mamba.ModelChains"

Iterations = 252:10000
Thinning interval = 2
Chains = 1,2,3
Samples per chain = 4875

4875x3x3 Array{Float64,3}:
[:, :, 1] =
  1.64512   0.590281  0.745324
  2.01987   0.21768   0.804604
  1.90365   0.197333  1.12632 
  1.83461   0.407171  0.861489
 -0.364267  1.0588    0.673022
 -0.232517  0.949028  0.596242
 -0.577513  1.19428   0.56121 
 -0.577513  1.19428   0.482966
 -0.75152   1.07638   0.444015
  1.20238   0.554745  0.542713
  1.01894   0.602312  0.565783
  1.09983   0.616616  0.331206
  1.26023   0.761922  0.797652
  ⋮                           
  0.138431  1.01351   0.864745
  0.307369  0.848039  0.27642 
  0.189732  0.850556  0.570012
 -0.430243  1.18742   0.765399
 -0.331263  1.27906   1.47324 
 -1.55257   1.21116   2.11123 
 -1.5476    1.21237   0.839099
  1.9984    0.345858  1.33285 
  1.77865   0.7871    2.66465 
  2.65552   0.350876  2.91149 
  1.86663   0.310991  2.9489  
 -2.19677   1.75209   1.44492 

[:, :, 2] =
  0.466503    0.8358

In [29]:
sim = mcmc(sim1, 5000)
describe(sim)

MCMC Simulation of 5000 Iterations x 3 Chains...

Chain 1:   0% [0:00:14 of 0:00:14 remaining]
Chain 1:  10% [0:00:04 of 0:00:04 remaining]
Chain 1:  20% [0:00:03 of 0:00:04 remaining]
Chain 1:  30% [0:00:03 of 0:00:04 remaining]
Chain 1:  40% [0:00:02 of 0:00:04 remaining]
Chain 1:  50% [0:00:02 of 0:00:04 remaining]
Chain 1:  60% [0:00:02 of 0:00:04 remaining]
Chain 1:  70% [0:00:01 of 0:00:05 remaining]
Chain 1:  80% [0:00:01 of 0:00:05 remaining]
Chain 1:  90% [0:00:00 of 0:00:04 remaining]
Chain 1: 100% [0:00:00 of 0:00:05 remaining]

Chain 2:   0% [0:00:06 of 0:00:06 remaining]
Chain 2:  10% [0:00:04 of 0:00:04 remaining]
Chain 2:  20% [0:00:03 of 0:00:04 remaining]
Chain 2:  30% [0:00:03 of 0:00:04 remaining]
Chain 2:  40% [0:00:03 of 0:00:04 remaining]
Chain 2:  50% [0:00:02 of 0:00:04 remaining]
Chain 2:  60% [0:00:02 of 0:00:04 remaining]
Chain 2:  70% [0:00:01 of 0:00:04 remaining]
Chain 2:  80% [0:00:01 of 0:00:04 remaining]
Chain 2:  90% [0:00:00 of 0:00:04 remaining]
Chai

In [31]:
## Default summary plot (trace and density plots)
p = plot(sim1)

## Write plot to file
draw(p, filename="summaryplot.svg")

In [32]:
## Autocorrelation and running mean plots
p = plot(sim1, [:autocor, :mean], legend=true)
draw(p, nrow=3, ncol=2, filename="autocormeanplot.svg")

In [33]:
## Pairwise contour plots
p = plot(sim1, :contour)
draw(p, nrow=2, ncol=2, filename="contourplot.svg")

In [34]:
## Development and Testing

setinputs!(model, line)             # Set input node values
setinits!(model, inits[1])          # Set initial values
setsamplers!(model, scheme1)        # Set sampling scheme

showall(model)                      # Show detailed node information

logpdf(model, 1)                    # Log-density sum for block 1
logpdf(model, 2)                    # Block 2
logpdf(model)                       # All blocks

sample!(model, 1)                  # Sample values for block 1
sample!(model, 2)                  # Block 2
sample!(model)                     # All blocks

Object of type "Mamba.Model"
-------------------------------------------------------------------------------
y:
An unmonitored node of type "5-element Mamba.ArrayStochastic{1}"
[1.0,3.0,3.0,3.0,5.0]

Distribution:
IsoNormal(
dim: 5
μ: [1.9283935989372833,3.2957024980833443,4.663011397229405,6.030320296375466,7.397629195521527]
Σ: [0.36645899740731525 0.0 0.0 0.0 0.0
 0.0 0.36645899740731525 0.0 0.0 0.0
 0.0 0.0 0.36645899740731525 0.0 0.0
 0.0 0.0 0.0 0.36645899740731525 0.0
 0.0 0.0 0.0 0.0 0.36645899740731525]
)

Function:
AST(:($(Expr(:lambda, Any[:(model::Model)], Any[Any[Any[:model,:Any,18],Any[:f,:Any,18]],Any[],0,Any[]], :(begin 
        model = (top(typeassert))(model,Mamba.Model)
        f = (anonymous function)
        return f((Mamba.getindex)(model,:mu),(Mamba.getindex)(model,:s2))
    end)))))

Source Nodes:
[:mu,:s2]

Target Nodes:
Symbol[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "Mamba.ScalarStochastic"

Object of type "Mamba.Model"
-------------------------------------------------------------------------------
y:
An unmonitored node of type "5-element Mamba.ArrayStochastic{1}"
[1.0,3.0,3.0,3.0,5.0]
-------------------------------------------------------------------------------
s2:
A monitored node of type "Mamba.ScalarStochastic"
0.20655200954079586
-------------------------------------------------------------------------------
xmat:
5x2 Array{Float64,2}:
 1.0  1.0
 1.0  2.0
 1.0  3.0
 1.0  4.0
 1.0  5.0
-------------------------------------------------------------------------------
beta:
A monitored node of type "2-element Mamba.ArrayStochastic{1}"
[0.46499943994841353,0.8254574189506928]
-------------------------------------------------------------------------------
mu:
An unmonitored node of type "5-element Mamba.ArrayLogical{1}"
[1.2904568588991063,2.1159142778497992,2.9413716968004917,3.7668291157511846,4.592286534701877]


α=0.001, θ=1000.0)
θ: 0.001
)

Function:
AST(:($(Expr(:lambda, Any[:(model::Model)], Any[Any[Any[:model,:Any,18],Any[:f,:Any,18]],Any[],0,Any[]], :(begin 
        model = (top(typeassert))(model,Mamba.Model)
        f = (anonymous function)
        return f()
    end)))))

Source Nodes:
Symbol[]

Target Nodes:
[:y]
-------------------------------------------------------------------------------
xmat:
[1.0 1.0
 1.0 2.0
 1.0 3.0
 1.0 4.0
 1.0 5.0]
-------------------------------------------------------------------------------
beta:
A monitored node of type "2-element Mamba.ArrayStochastic{1}"
[0.5610846997912224,1.367308899146061]

Distribution:
ZeroMeanIsoNormal(
dim: 2
μ: [0.0,0.0]
Σ: [1000.0 0.0
 0.0 1000.0]
)

Function:
AST(:($(Expr(:lambda, Any[:(model::Model)], Any[Any[Any[:model,:Any,18],Any[:f,:Any,18]],Any[],0,Any[]], :(begin 
        model = (top(typeassert))(model,Mamba.Model)
        f = (anonymous function)
        return f()
    end)))))

Source Nodes:
Symbol[]

Target Nodes